# Proyecto 1 de programación
**Atzi Merino**

Imports

In [13]:
import numpy as np
import pandas as pd

Clases

In [14]:
#Esta clase guarda los metodos y atributos de una graficas
class Grafica:
  def __init__(self, nodos, arcos):
    #Se le asocia una matriz cuadrada
    self.matriz = np.zeros((len(nodos),len(nodos)))
    #Este for modifica la matriz para que represente la incidencia de arcos
    for x in arcos:
      self.matriz[x[0],x[1]] = 1
  #Tiene un metodo str para que al imprimir se vea bien el objeto
  def __str__(self):
    return str(self.matriz)

In [15]:
#Esta clase hereda de una grafica y representa una red de sitios web
class Red(Grafica):
  def __init__(self, ruta_del_archivo):
    #Es necesario un archivo con los datos de los sitios
    self.ruta_del_archivo = ruta_del_archivo
  #Este metodo forma los arcos y los nodos y los guarda en una grafica
  def construccion_pd(self):
    df = pd.read_excel(self.ruta_del_archivo)
    self.nodos = (list(range(len(df["Index"]))))
    self.arcos = []
    citados = list(df["Cited by"])
    for i in range(len(citados)):
      for j in citados[i].split(','):
        self.arcos.append((int(j)-1,i))
    return Grafica(self.nodos, self.arcos)
  #Este metodo extrae las paginas web del acrhivo
  def paginas_web(self):
    df = pd.read_excel(self.ruta_del_archivo)
    paginas_web = list(df["Website"])
    return paginas_web

Funciones

In [16]:
#Esta funcion hace que los renglones de la matriz sumen 1
def arreglo_renglones(matriz):
  for i in range(len(matriz)):
    matriz[i,: ] = matriz[i,: ] / sum(matriz[i,: ])
  return matriz

In [17]:
#Esta funcion forma un vector de entradas 1/n
#Su tamaño dependerá del tamaño de matriz con la que trabajamos
def vector_1_n(matriz):
  vector = np.zeros((1, len(matriz)))
  for i in range(len(matriz)):
    vector[0, i] = 1/len(matriz)
  return vector

In [18]:
#Esta funcion construye una matriz con entradas 1/n
def matriz_1_n(matriz):
  matriz1n = np.zeros((len(matriz), len(matriz)))
  for i in range(len(matriz)):
    for j in range(len(matriz)):
      matriz1n[i, j] = 1/len(matriz)
  return matriz1n

In [19]:
#Esta funcion forma un vector dada una terminación de sitio web
def vector_terminacion(red, matriz, terminacion):
  vector = np.zeros((1, len(matriz)))
  contador = 0
  for i in range(len(matriz)):
    if red.paginas_web()[i].endswith(terminacion):
      vector[0, i] = 1
      contador = contador + 1
    else:
      vector[0,i] = 0
  return (1/contador)*vector

In [20]:
#Esta función multiplica iterativamente el vector y la matriz
def page_rank(matriz,vector):
  vector_resultante = vector
  for i in range(10000):
    vector_resultante = np.matmul(vector_resultante,matriz)
  return vector_resultante

In [21]:
#Esta función multiplica iterativamente añadiendo la u y la d
def page_rank_u(matriz, vector, d):
  u = matriz_1_n(matriz)
  vector_resultante = vector
  for i in range(10000):
    vector_resultante = np.matmul(vector_resultante, d * matriz + (1-d) * u)
  return vector_resultante

In [22]:
#Esta funcion devuelve una lista con las importancias ordenadas
def importancia_paginas(vector_importancia):
  lista_importancias = []
  tuplas = []
  paginas = list(range(np.size(vector_importancia)))
  #Primero se convierte el vector en lista
  for i in range(np.size(vector_importancia)):
    lista_importancias.append(vector_importancia[0, i])
  #Luego generé una lista de tuplas de la importancia y la página web asociada
  for i in range(len(lista_importancias)):
    tuplas.append((lista_importancias[i],paginas[i]+1))
  #Las ordeno de menor a mayor
  importancias_ordenadas = sorted(tuplas)
  #Luego lo invierto para que el mayor quede primero
  importancias_ordenadas.reverse()
  return importancias_ordenadas

Main

In [23]:
#La función main usa todas las anteriores para contestar las preguntas
def main():
  red_paginas = Red("/content/Web.xlsx")
  matriz_citas = red_paginas.construccion_pd().matriz
  p = arreglo_renglones(matriz_citas)
  #Pregunta 1
  vector1n = vector_1_n(p)
  vector_importancia = page_rank(p,vector1n)
  importancias_1 = importancia_paginas(vector_importancia)
  #Pregunta 2
  vector_ru = vector_terminacion(red_paginas,p,".ru")
  vector_importancia_ru = page_rank(p,vector_ru)
  importancias_2 = importancia_paginas(vector_importancia_ru)
  #Pregunta 3
  d1 = 0.5
  vector_importancia_d1 = page_rank_u(p,vector1n,d1)
  importancias_d1 = importancia_paginas(vector_importancia_d1)

  d2 = 0.85
  vector_importancia_d2 = page_rank_u(p,vector1n,d2)
  importancias_d2 = importancia_paginas(vector_importancia_d2)

  d3 = 1
  vector_importancia_d3 = page_rank_u(p,vector1n,d3)
  importancias_d3 = importancia_paginas(vector_importancia_d3)

  print("la página más importante iniciando con el vector (1/n,..,1/n) es la "
        + str(importancias_1[0][1]))
  print("la página más importante tomando sitios web .ru es la "
        + str(importancias_2[0][1]))
  print("la página más importante agregado la u y tomando d="
        + str(d1) + " es la "  + str(importancias_d1[0][1]))
  print("la página más importante agregado la u y tomando d="
        + str(d2) + " es la "  + str(importancias_d2[0][1]))
  print("la página más importante agregado la u y tomando d="
        + str(d3) + " es la "  + str(importancias_d3[0][1]))

In [24]:
if __name__ == '__main__':
  main()

la página más importante iniciando con el vector (1/n,..,1/n) es la 26
la página más importante tomando sitios web .ru es la 24
la página más importante agregado la u y tomando d=0.5 es la 26
la página más importante agregado la u y tomando d=0.85 es la 26
la página más importante agregado la u y tomando d=1 es la 26
